In [1]:
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import os
import sys
import pickle
import json

from data_prov import *
from model import *
from options import *



In [2]:
img_home = '../../tracking_benchmark/dataset/'
data_path = 'data/vot-otb.pkl'
VID_data_path = 'data/ilsvrc_train_500.json'
VID_home = '../../tracking_benchmark/ILSVRC2015/Data/VID/train/'

In [3]:
writer = SummaryWriter()

In [4]:
    with open(VID_data_path, 'r') as f:
        data = json.load(f)

    K = len(data)
    dataset = [None]*K
    for k, seq in enumerate(iter(data)):
        seq_name = seq['seq_name']
        start_frame = seq['start_frame']
        end_frame = seq['end_frame']
        gt = np.array(seq['gt'])
        im_width = seq['im_width']
        im_height = seq['im_height']
        img_dir = os.path.join(VID_home, seq_name)
        dataset[k] = ILSVRC2015RegionDataset(img_dir, start_frame, end_frame, gt, im_width, im_height, opts)



In [5]:
opts['init_model_path'] = '../models/pymdnet-vot-attmask2-conv1&se3&loss3-1024.pth'
print(opts['init_model_path'])
print(K)

../models/pymdnet-vot-attmask2-conv1&se3&loss3-1024.pth
500


In [6]:
model = MDNet(opts['init_model_path'], K)
model = model.cuda()
model = model.eval()

conv3_se_weight have already in param, so plus _2
conv3_se_bias have already in param, so plus _2


In [7]:
label = torch.IntTensor(160, 1)
label[:32] = 1
label[32:128] = 0
label[128:]=2
label.tolist()

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2]]

In [8]:

## for 1

k = 0
pos_regions, neg_regions = dataset[k].next()
pos_regions = Variable(pos_regions)
neg_regions = Variable(neg_regions)

pos_regions = pos_regions.cuda()
neg_regions = neg_regions.cuda()

p_features = model(pos_regions, k, out_layer='fc5')
n_features = model(neg_regions, k, out_layer='fc5')
total_features = torch.cat((p_features, n_features), 0)
print(total_features.size())

## for 2
k = 10
pos_regions, neg_regions = dataset[k].next()
pos_regions = Variable(pos_regions)
neg_regions = Variable(neg_regions)

pos_regions = pos_regions.cuda()
neg_regions = neg_regions.cuda()

p_features = model(pos_regions, k, out_layer='fc5')
total_features = torch.cat((total_features, p_features), 0)
print(total_features.size())


torch.Size([128, 512])
torch.Size([160, 512])


In [9]:
writer.add_embedding(total_features.data.cpu(), metadata=label)
writer.export_scalars_to_json('./all_scalars.json')
writer.close()